# Redis SQL to RedisJSON + RediSearch Demo

This notebook demonstrates how to represent SQL-like relational data (Users, Portfolio, Transactions) using RedisJSON, index it with RediSearch, and query using RediSearch DSL.

In [54]:
import redis
import json
import random
from datetime import datetime, timedelta

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Insert dummy data
for i in range(1, 11):
    user_id = f"u{i}"
    portfolio = [
        {"stock": "AAPL", "quantity": random.randint(1, 20), "gainLoss": round(random.uniform(-10, 25), 2)},
        {"stock": "TSLA", "quantity": random.randint(1, 10), "gainLoss": round(random.uniform(-15, 30), 2)},
        {"stock": "HDFB Bank", "quantity": random.randint(1, 20), "gainLoss": round(random.uniform(-10, 25), 2)},
        {"stock": "Maxis Bank", "quantity": random.randint(1, 10), "gainLoss": round(random.uniform(-15, 30), 2)},
        {"stock": "TAHATA", "quantity": random.randint(1, 20), "gainLoss": round(random.uniform(-10, 25), 2)},
        {"stock": "JIJO", "quantity": random.randint(1, 10), "gainLoss": round(random.uniform(-15, 30), 2)}
    ]
    transactions = [
        {
            "txnId": f"t{i}_{j}",
            "type": random.choice(["BUY", "SELL"]),
            "amount": random.randint(500, 5000),
            "timestamp": (datetime.now() - timedelta(days=j)).isoformat()
        }
        for j in range(11)
    ]
    doc = {
        "userId": user_id,
        "name": f"User {i}",
        "portfolio": portfolio,
        "transactions": transactions
    }
    r.json().set(f"user:{user_id}", "$", doc)

print("Sample user documents inserted.")

Sample user documents inserted.


In [52]:
from redis.commands.search.field import TextField, NumericField
from redis.commands.search.index_definition import IndexDefinition, IndexType
from redis.commands.search import Search

try:
    r.ft("idx:users").dropindex(delete_documents=False)
except:
    pass
# Define schema
schema = (
    TextField("$.userId", as_name="userId"),
    TextField("$.portfolio[*].stock", as_name="stock"),
    NumericField("$.portfolio[*].gainLoss", as_name="gainLoss"),
    NumericField("$.transactions[*].amount", as_name="amount"),
    TextField("$.transactions[*].timestamp", as_name="timestamp")
)

# Define index definition (use IndexType.JSON if using RedisJSON)
definition = IndexDefinition(prefix=["user:"], index_type=IndexType.JSON)

# Create index
r.ft("idx:users").create_index(fields=schema, definition=definition)


print("idx:users Index created.")

idx:users Index created.


In [60]:
from redis.commands.search.query import Query

user_id = "u1"
q = Query(f"@userId:{user_id}").return_fields("$.portfolio").sort_by("gainLoss", asc=False).dialect(2)
res = r.ft("idx:users").search(q)

for doc in res.docs:
    print(f"\nDocument ID: {doc.id}")
    
    raw_portfolio = doc["$.portfolio"]
    try:
        # Parse the JSON string into a Python list
        portfolio = json.loads(raw_portfolio)
        print("Parsed Portfolio:")
        for item in portfolio:
            print(f"  • Stock: {item['stock']} | Quantity: {item['quantity']} | Gain/Loss: {item['gainLoss']}")
    except Exception as e:
        print("Error parsing portfolio:", e)
        print("Raw:", raw_portfolio)

In [59]:
q = Query(f"@userId:{user_id}").return_fields("$.transactions").sort_by("timestamp", asc=False).dialect(2)
res = r.ft("idx:users").search(q)

for doc in res.docs:
    try:
        transactions = json.loads(doc["$.transactions"])
        print("Transactions:")
        for tx in transactions:
            print(f"  • txnId: {tx['txnId']} | Date: {tx['timestamp']} | Type: {tx['type']} | Amount: {tx['amount']}")
    except Exception as e:
        print("Error parsing transactions:", e)